ここでは，町田市内のイタリアンレストランから一定距離内にある人口を計算する。

まず，レストランの位置を記述したファイルから，住所を読み込む。そして，それらの所在地に対してPointを生成し，その結果をgeojsonファイルとして保存する。



In [1]:
import pandas as pd 
data=pd.read_csv("data/italian_rest_machida.txt",sep=";")

In [2]:
data

,id,name,addr
0,0,minami-naruse,東京都町田市南成瀬2-7-1
1,1,kanai,東京都町田市金井ケ丘2-24-16
2,2,tadao,東京都町田市木曽西2-17-20
3,3,tips-machida,東京都町田市原町田6-7-8
4,4,tsurukawa,東京都町田市能ヶ谷1-3-2


In [3]:
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point

geolocator = Nominatim(user_agent="test-dayo")
data['gcode']=data.addr.apply(geolocator.geocode)


In [ ]:
data['coords']=[(g.longitude,g.latitude) for g in data['gcode']] 
data['geometry']=data['coords'].apply(Point)

In [ ]:
data

In [ ]:
gdata=gpd.GeoDataFrame(data)
gdata.set_crs("EPSG:6668")


In [ ]:
gdata['geometry'].plot()

In [ ]:
tokyofp="data/N03-190101_13_GML/N03-19_13_190101.geojson"
tokyo=gpd.read_file(tokyofp)
tokyo.columns

In [ ]:
machida=tokyo[tokyo["N03_004"]=="町田市"]
machida=machida.reset_index(drop=True)
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,1)
machida.boundary.plot(ax=ax)
gdata['geometry'] = gdata['geometry'].set_crs("EPSG:6668")
gdata['geometry'].plot(ax=ax)

In [ ]:
machida

人口データをみよこむ。　各カラムの表すデータは次のものである。

MESH_ID - - - メッシュコード

SHICODE 2018年 - - 市町村コード

PTN_2015 2015年 男女計 総数 人口

PMN_2015 2015年 男 総数 人口

PFN_2015 2015年 女 総数 人口

PTN_2020 2020年 男女計 総数 人口（秘匿なし）

In [ ]:
popfp="data/500m_mesh_suikei_2018_shape_13/500m_mesh_2018_13.shp" #人口データ
pop=gpd.read_file(popfp)
pop['geometry'] = pop['geometry'].to_crs("EPSG:6668")
machida_pop=pop[pop['SHICODE']==13209] #町田の市町村コード13209

In [ ]:
machida_pop

読み込んだデータの各行は，1つのメッシュデータを表すが，列としてgeometryを持っている。したがって，表すメッシュをプロットして可視化することができる。

In [ ]:
machida_pop['geometry'].boundary.plot(color="red",alpha=0.4)

In [ ]:
fig,ax=plt.subplots(1,1)
machida.boundary.plot(ax=ax)
machida_pop['geometry'].boundary.plot(ax=ax,color="red",alpha=0.4)
gdata['geometry'].plot(ax=ax,color="green")

ここでは，500mメッシュ別の人口を用いる。データには，メッシュコードを表す列MESH_IDがある。これは，ある地域を表すコードである。このコードは，表そうとする地域の緯度経度を元に設定されている。

523867884 というコードを例とする。

https://www.stat.go.jp/data/mesh/m_tuite.html

https://home.csis.u-tokyo.ac.jp/~nishizawa/teikyo/chiiki_mesh.pdf


地域メッシュは、緯線・経線によって囲まれたほぼ長方形の区画である。経度（東西方向）1 度、緯度（南北方向）40 分の範囲を１次メッシュという。メッシュのコード番号はメッシュの南西隅の「緯度×3/2」（2 桁）＋「経度－100」（2 桁）の 4 桁の数値とされている。 例えば，　町田市役所は北緯35.55度，東経139.44度である。35.55x1.5=53.32, 139.44-100=39.4なので， コードの最初の４桁は， 5339となる。

> １次メッシュを縦横 8 等分したものを２次メッシュ（約 10km 四方）、２次メッシュを縦横 10 等分したものを３次メッシュ（約 1km 四方）という。３次メッシュは 1km メッシュとも呼ばれる。メッシュのコード番号は、１次メッシュのコード番号に、メッシュを分割するごとに、縦方向の番号と横方向の番号を付け足していく。２次メッシュは 6 桁、３次メッシュは 8 桁になる。下の図の右上の３次メッシュの番号は 53393393 となる。３次メッシュまでは JIS で定められている。 

4次メッシュが500mメッシュに対応する。これは，3次メッシュをさらに緯度，経度方向に２等分したものである。

このプロットを観察すると，町田市内の店舗は，市境に立地しているものが多く，市外からの顧客も見込める。そこで，町田に隣接する都市も加えることにする。やはり神奈川も要るので神奈川の行政区域を表すファイルを読み込む。

このためには，データを町田市のものに限るのをやめて，東京都と神奈川県の全体のデータを用いる。

In [ ]:
kanagawafp="data/N03-190101_14_GML/N03-19_14_190101.geojson"
kanagawa=gpd.read_file(kanagawafp)
tokyofp="data/N03-190101_13_GML/N03-19_13_190101.geojson"
tokyo=gpd.read_file(tokyofp)

In [ ]:
islands = ['大島町', '利島村', '新島村', '神津島村', '三宅村', '御蔵島村', '八丈町', '青ヶ島村', '小笠原村','所属未定地']
is_not_islands=[tokyo['N03_004']!=island for island in islands]
is_not_islands=pd.concat(is_not_islands,axis=1).all(axis=1)
tokyo=tokyo.loc[is_not_islands,:]
tokyo['N03_004'].unique()

In [ ]:
tokyo.plot()

In [ ]:
kanagawa.plot()

東京都と神奈川を合わせて解析するので，2つのDataFrame, tokyo, kanagawaを結合する。

In [ ]:
tokyo_kanagawa=pd.concat([tokyo,kanagawa])

In [ ]:
tokyo_kanagawa.plot()

同様に，東京と神奈川の人口データを読み込み結合する。

In [ ]:
tokyo_popfp="data/500m_mesh_suikei_2018_shape_13/500m_mesh_2018_13.shp" #人口データ
tokyo_pop=gpd.read_file(tokyo_popfp)
tokyo_pop['geometry'] = tokyo_pop['geometry'].to_crs("EPSG:6668")

kana_popfp="data/500m_mesh_suikei_2018_shape_14/500m_mesh_2018_14.shp" #人口データ
kana_pop=gpd.read_file(kana_popfp)
kana_pop['geometry']=kana_pop['geometry'].to_crs("EPSG:6668")
tokyo_kana_pop=pd.concat([tokyo_pop,kana_pop])
tokyo_kana_pop=tokyo_kana_pop.to_crs("EPSG:6668")

In [ ]:
fig,ax=plt.subplots(1,1)
gdata['geometry'].plot(ax=ax, color="green")
tokyo_kana_pop.plot(ax=ax)

ここでは，gdataに含まれるレストラン5件の（最大で）周囲２０kmに興味があるので，この地域に含まれるメッシュを取り出す。これには，unary_unionを用いてレストラン５件の和集合を求め，さらにその凸包を求める。

In [ ]:


gdata_convex_hull=gdata['geometry'].unary_union.convex_hull
gdata_convex_hull=gpd.GeoSeries(gdata_convex_hull)
gdata_convex_hull=gdata_convex_hull.set_crs(gdata['geometry'].crs)
gdata_convex_hull.plot()

そうして，この凸包のバッファを設定する。バッファを設定するには，距離を用いる必要があるので，投影を用いた座標系（ここでは3857）に変換してからバッファを計算する。求めたバッファを，東京都・神奈川県の地図に重ねてプロットする。

In [ ]:
gdata_convex_hull=gdata_convex_hull.to_crs("EPSG:3857")
gdata_convex_hull_buf=gdata_convex_hull.buffer(1*1e+3)
gdata_convex_hull=gdata_convex_hull.to_crs("EPSG:6668")
gdata_convex_hull_buf=gdata_convex_hull_buf.to_crs("EPSG:6668")

fig,ax=plt.subplots(1,1)
gdata_convex_hull_buf.boundary.plot(ax=ax,color="red")
gdata_convex_hull.boundary.plot(ax=ax,color="green")
machida.boundary.plot(ax=ax)
gdata.plot(ax=ax,color="gray")

    バッファと共通部分を持つメッシュのみを取り出して表示する。これには，spatial join演算を用いる。

In [ ]:
print(gdata.crs,gdata_convex_hull_buf.crs,machida.crs,tokyo_kana_pop.crs)

In [ ]:


tokyo_kana_pop=tokyo_kana_pop.set_crs("EPSG:6668")
gdata_convex_hull_buf_df=gpd.GeoDataFrame() #バッファ付きの凸包をDataFrameとする。
gdata_convex_hull_buf_df.loc[0,'geometry']=gdata_convex_hull_buf[0]
gdata_convex_hull_buf_df=gdata_convex_hull_buf_df.set_crs(tokyo_kana_pop.crs)

In [ ]:
#gdata_convex_hull_buf_df.plot() #バッファつきの凸包をプロットする。
# 東京と神奈川の人口メッシュデータの中て，　凸包と共通部分を持つものをmesh_around_restとして取り出す。
mesh_around_rest=gpd.sjoin(tokyo_kana_pop,gdata_convex_hull_buf_df,how='inner',op='intersects')
fig,ax=plt.subplots(1,1)
mesh_around_rest.boundary.plot(ax=ax,color='red')
gdata['geometry'].plot(ax=ax)

 ここまでで，5つのレストランからおよそ10km圏内にある人口メッシュmesh_around_restを取り出すことができた。
 次に，5つのレストランそれぞれについて，およそ10km圏内の人口メッシュを取り出す。
 
 まずは，gdataに格納されている5つのレストランの場所を表すPointのバッファを計算し，gdata['buffer']に記録する。

In [ ]:
#各レストランのバッファを計算して'buffer'列に保存する。
gdata=gdata.to_crs("EPSG:3857") ;
gdata['buffer']=gdata.buffer(1*1e+3);
gdata=gdata.to_crs("EPSG:6668");

fig,ax=plt.subplots(1,1);
gdata['buffer']=gdata['buffer'].to_crs("EPSG:6668");
fig,ax=plt.subplots(1,1);
#gdata['buffer'].boundary.plot(ax=ax); #バッファをプロットする
#gdata.plot(ax=ax); #レストランの所在地のプロット
gdata_buf_df=gpd.GeoDataFrame(geometry=gdata['buffer'])
tmesh_around_rest=gpd.sjoin(tokyo_kana_pop,gdata_buf_df,how='inner',op='intersects')

gdata_buf_df.boundary.plot()

tmesh_around_rest.plot()

In [ ]:
mesh_around_rest=gpd.sjoin(tokyo_kana_pop,rch_buf_df,how='inner',op='intersects')
fig,ax=plt.subplots(1,1)
machida.boundary.plot(ax=ax)


町田市に隣接するのは，東京都多摩市，東京都稲城市，東京都八王子市，神奈川県相模原市，神奈川県川崎市，神奈川県大和市，座間市

https://github.com/MIERUNE/japan-mesh-tool

In [ ]:
tmesh=gpd.read_file("tmp/mesh_2.geojsonl")
tmesh=tmesh.to_crs("EPSG:6668")
tmesh

これを，町田市の地図，レストランの位置と合わせてプロットする。

In [ ]:
fig,ax=plt.subplots(1,1)
machida.boundary.plot(ax=ax)
gdata['geometry'].plot(ax=ax)
tmesh.boundary.plot(ax=ax,color="red")

In [ ]:
machida_pop['geometry'].plot()

人口データは，メッシュに対して与えられている。ここで，レストランの位置から周囲１０km以内の人口を考察の対象とすることにする。このためには，メッシュコードを手がかりに，　レストランの位置から周囲10km以内にあるメッシュを特定したい。上記のgithubリポジトリで，緯度経度から，該当するメッシュを計算するプログラムが公開されているので，それを用いて2次メッシュ(約10km四方)を求めることとする。

いま，例として， minami-naruse (139.4669433, 35.5396126)を用いる。上記のjapan-mesh-toolの使い方は，次のとおりである。コマンドラインで

```bash
python japan-mesh-tool-master/python/japanmesh/main.py -e 139.4669433,35.5396126 139.47,35.540 -d ./tmp 2 

```

とすると， -eで指定した2つの緯度経度を左下と右上にする２次のメッシュを表すobjectが, geojsonファイルとして， tmpフォルダに出力される。2次メッシュを計算することは，最後の引数で指定している。

こうして出力されたファイルの中身は次のものである。

```
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[139.375, 35.5], [139.375, 35.58333333333333], [139.5, 35.58333333333333], [139.5, 35.5], [139.375, 35.5]]]},"properties":{"code":533923}}
```

これより，求めたかった2次メッシュのコードは， 533923であることがわかる。